In [1]:
import pandas as pd;
import numpy as  np;
from scipy.optimize import minimize;
import random;

In [2]:
ratings_small = np.genfromtxt('ml-latest-small//ratings.csv',delimiter=',',skip_header =1)[:,0:3]

In [3]:
def get_rhat(u,v):
    return np.matmul(u,np.transpose(v))

In [4]:
#min J= 1/2||R - Utranspose(V)||**2
###Calculate reconstruction error

def cal_error(r,rhat):
    return 0.5 * np.sum(np.square(r - rhat))    

In [99]:
def matrix_factorization(ratings,alpha,steps,concept=3):
    r=ratings
    row = (r.shape[0])
    col = (r.shape[1])
    
    np.random.seed(1234)
    u = np.random.rand(row,concept)
    v = np.random.rand(col,concept)
    
    e = r - get_rhat(u,v)
    e = np.array(pd.DataFrame(e).fillna(0))
    #e1 = np.zeros((row,col))
    #
    #for i in range(row):
    #    for j in range(col):
    #        if r[i][j] > 0:
    #            e1[i,j] = r[i][j] - get_rhat(u,v)[i,j]
    #           
               
    #e = r - get_rhat(u,v)
    #print(e)
    #print(e1)
    for s in range(steps):
        u_c = u + alpha * np.matmul(e,v)
        v_c = v + alpha * np.matmul(np.transpose(e),u)
       
        u = np.copy(u_c)
        v = np.copy(v_c)
        e = r - get_rhat(u,v)
        e = np.array(pd.DataFrame(e).fillna(0))
    return get_rhat(u,v)

In [100]:
R = [
  [5,3,np.nan,1],
  [4,np.nan,np.nan,1],
  [1,1,np.nan,5],
  [1,np.nan,np.nan,4],
  [np.nan,1,5,4],
 ]
R = np.array(R)


In [101]:
matrix_factorization(R,0.002,1000)

array([[5.00479217, 2.98757696, 3.29128869, 0.99876123],
       [3.99561129, 2.12559217, 3.00200174, 0.99823669],
       [1.01425366, 0.96503099, 4.64540656, 4.997607  ],
       [0.99271356, 0.4237421 , 4.14724211, 3.99633337],
       [2.37957614, 1.04454135, 4.98434807, 4.01184795]])